<a href="https://colab.research.google.com/github/raffieeey/MasterResearchAutoML/blob/master/04A_Autosklearn_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install gcc
!gcc --version
!apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn
!pip install pmlb
!pip install pytictoc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
gcc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/unive

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import math
import numpy as np

class MetricsCalc:

    def __init__(self, truth, predictions):
        self.truth = pd.DataFrame(truth)
        self.predictions = predictions
        self._type = 'regression'

    def mae(self):
        return float(mean_absolute_error(self.truth, self.predictions))

    def mse(self):
        return float(mean_squared_error(self.truth, self.predictions))

    def msle(self):
      try:
        return float(mean_squared_log_error(self.truth, self.predictions))
      except:
        return np.nan
    
    def rmse(self):
        return math.sqrt(self.mse())

    def rmsle(self):
        return math.sqrt(self.msle())

    def r2(self):
        return float(r2_score(self.truth, self.predictions))

class CheckNegativeValue:
  def __init__(self, checknegative):
    self._is_negative = "neg" if np.min(checknegative) < 1 else "pos"
  
  def MLMetrics(self):
    if self._is_negative == "neg" :
      return [metrics.mean_absolute_error, metrics.mean_squared_error, metrics.r2]
    elif self._is_negative == "pos" :
      return ['neg_mean_squared_log_error', metrics.mean_absolute_error, metrics.mean_squared_error, metrics.r2]

In [45]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from autosklearn.estimators import AutoSklearnRegressor
import autosklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from pytictoc import TicToc
import pandas as pd
from pprint import pprint

df_result = pd.DataFrame(columns=["dataset_name", "metrics_recorded", "MAE", "MSE","MSLE", "RMSE", "RMSLE", "r2"])

preds = {}
cm_preds = {}
top_algo = {}
t = TicToc()


sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

ix = 0

for rgrs_dtst in sel_rgrs_dtst:
  for metrics_recorded  in [metrics.mean_absolute_error, metrics.mean_squared_error, metrics.r2, metrics.median_absolute_error]:
        print("_"*80)
        print(f"*** Dataset Name: {rgrs_dtst} ***")
        df_result.loc[ix,"dataset_name"] = rgrs_dtst 
        df_result.loc[ix,"metrics_recorded"] = metrics_recorded 
        X, y = fetch_data(rgrs_dtst , return_X_y=True)
        c=CheckNegativeValue(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        y_train = pd.DataFrame(y_train).values.ravel()
        y_test = pd.DataFrame(y_test).values.ravel()
        list_metrics = c.MLMetrics()
        automl = AutoSklearnRegressor(seed=69, n_jobs=2, ml_memory_limit=2048,time_left_for_this_task=120)
        t.tic()
        automl.fit(X_train,y_train, metric=metrics_recorded if metrics_recorded in list_metrics else None)
        t.toc()
        inse = automl.show_models()
        try:
          res_json=eval(inse.split("(")[2].split("))")[0].split(",\ndataset_properties")[0])
        except:
          print('dummy_regressor')
        top_algo[rgrs_dtst] = res_json['regressor:__choice__']
        print("*** Top algorithm: ***", res_json['regressor:__choice__'])
        pprint(res_json)
        preds[rgrs_dtst] = automl.predict(X_test)
        p = MetricsCalc(y_test,preds[rgrs_dtst])
        df_result.loc[ix,'MAE'] = p.mae()
        df_result.loc[ix,'MSE'] = p.mse()
        df_result.loc[ix,'MSLE'] = p.msle()
        df_result.loc[ix,'RMSE'] = p.rmse()
        df_result.loc[ix,'RMSLE'] = p.rmsle()
        df_result.loc[ix,'r2'] = p.r2()
        ix+=1

________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.603977)
[WARNING] [2020-05-08 15:42:00,941:AutoMLSMBO(1272380470)::76c127f2259c67c8487eca3f68a8aaa5] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/mean_absolute_error_regression_dense
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.573880)
Elapsed time is 115.065649 seconds.
*** Top algorithm: *** liblinear_svr
{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer',
 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.575350)
[WARNING] [2020-05-08 16:14:55,296:AutoMLSMBO(1272380470)::0c961f3c15e0da980826e52146e10f41] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/mean_squared_error_regression_dense
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.549030)
Elapsed time is 114.514988 seconds.
dummy_regressor
*** Top algorithm: *** adaboost
{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent',
 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer',
 'data_preprocessing:numerical_t

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.585310)
[WARNING] [2020-05-08 16:16:50,393:AutoMLSMBO(1272380470)::0c961f3c15e0da980826e52146e10f41] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/r2_regression_dense
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.565657)
Elapsed time is 114.491284 seconds.
dummy_regressor
*** Top algorithm: *** adaboost
{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent',
 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer',
 'data_preprocessing:numerical_transformer:resca

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.564172)
[WARNING] [2020-05-08 16:18:45,428:AutoMLSMBO(1272380470)::0c961f3c15e0da980826e52146e10f41] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/r2_regression_dense
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.550725)
Elapsed time is 114.517732 seconds.
dummy_regressor
*** Top algorithm: *** adaboost
{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent',
 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer',
 'data_preprocessing:numerical_transformer:resca

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.592870)
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.552440)
[WARNING] [2020-05-08 16:20:40,703:AutoMLSMBO(1272380470)::257ff3b2e2287efa7a29abfa06255ab1] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/mean_absolute_error_regression_dense
Elapsed time is 114.656018 seconds.
*** Top algorithm: *** adaboost
{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent',
 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer',
 'data_preprocessing:numerical_transformer:resc

In [46]:
for i, val in enumerate(preds):
  print(preds[val])

[6.44104275 1.54011543 2.26163761 5.10825937 5.03886608 4.33820466
 1.61314745 1.61274524 4.73898932 2.32570147 6.08629825 2.86641139
 4.84029438 2.50413451 2.17588837 5.60984449 2.17078892 4.94935988
 2.44652467 3.60852771 4.36137446 6.39599264 1.99782435 2.6385251
 1.78806352 3.5416007  5.75919459 2.61220043 5.92421878 4.54234649
 3.71461502 1.70904331 2.06757615 4.73318455 5.46639308 4.26196583
 3.67729735 2.23380154 2.93217315 2.1414236  4.96486188 4.76440138
 2.14363525 5.2770834  1.97551512 2.73100308 4.45079095 6.38628112
 2.4875835  4.72387274 5.18546278 2.40412655 2.75091417 3.45930163
 4.339      5.39210545 4.65222627 2.36391682 4.41215549 4.64616847
 3.87752645 1.82699508 2.38783691 6.20332397 1.75851642 6.12946614
 2.18405382 4.41276824 2.67668644 4.07973934 2.53808584 2.15318469
 4.5256327  5.01531508 6.0151321  2.41263181 2.09835209 5.33983704
 6.46530225 6.08983112 3.42509084 6.0022181  2.28856446 4.16648195
 5.86162658 6.28020506 3.4120671  6.2703756  4.64932216 3.13084

In [51]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pprint(df_result)

           dataset_name       metrics_recorded       MAE          MSE  \
0   294_satellite_image  mean_absolute_error    1.03535   1.4212        
1   294_satellite_image  mean_squared_error     1.05389   1.42432       
2   294_satellite_image  r2                     1.05389   1.42432       
3   294_satellite_image  median_absolute_error  1.05389   1.42432       
4   218_house_8L         mean_absolute_error    24439.2   1.58348e+09   
5   218_house_8L         mean_squared_error     24439.2   1.58348e+09   
6   218_house_8L         r2                     24439.2   1.58348e+09   
7   218_house_8L         median_absolute_error  24439.2   1.58348e+09   
8   227_cpu_small        mean_absolute_error    2.7308    11.358        
9   227_cpu_small        mean_squared_error     2.7308    11.358        
10  227_cpu_small        r2                     2.7308    11.358        
11  227_cpu_small        median_absolute_error  2.7308    11.358        
12  503_wind             mean_absolute_error    2.7

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.
